## CoNLL-2003 Example for Text Extensions for Pandas
### Part 2

To run this notebook, you will need to obtain a copy of the CoNLL-2003 data set's corpus.
Drop the corpus's files into the following locations:
* conll_03/eng.testa
* conll_03/eng.testb
* conll_03/eng.train

If you are unfamiliar with the basics of Text Extensions for Pandas, we recommend you 
start with Part 1 of this example.

### Introduction

In Part 1 of this demo, we showed how to use Text Extensions for Pandas to examine the 
overall result quality of one entrant in the CoNLL-2003 Shared Task, as well as how to
identify and examine the documents from the validation set where that entry had the 
most errors.

In Part 2, we'll perform a broader analysis that goes across all the contents entries
and come to some deeper and more surprising conclusions.

In [ ]:
# INITIALIZATION BOILERPLATE

# The Jupyter kernel for this notebook usually starts up inside the notebooks
# directory, but the text_extensions_for_pandas package code is in the parent
# directory. Add that parent directory to the front of the Python include path.
import sys
if (sys.path[0] != ".."):
    sys.path[0] = ".."
    
# Libraries
import numpy as np
import pandas as pd

# And of course we need the text_extensions_for_pandas library itself.
import text_extensions_for_pandas as tp

In [ ]:
# Load up the same gold standard data we used in Part 1.
gold_standard = tp.conll_2003_to_dataframes("../conll_03/eng.testa")
gold_standard_spans = [tp.iob_to_spans(df) for df in gold_standard]

In [ ]:
# Load up the results from all 16 teams at once.
teams = ["bender", "carrerasa", "carrerasb", "chieu", "curran",
         "demeulder", "florian", "hammerton", "hendrickx",
         "klein", "mayfield", "mccallum", "munro", "whitelaw",
         "wu", "zhang"]

# Read all the output files into one dataframe per <document, team> pair.
outputs = { 
    t: tp.conll_2003_output_to_dataframes(
        gold_standard, f"../resources/conll_03/ner/results/{t}/eng.testb")
    for t in teams
}  # Type: Dict[str, List[pd.DataFrame]]

# As an example of what we just loaded, show the token metadata for the 
# "mayfield" team's model's output on document 3.
outputs["mayfield"][3]

In [ ]:
# Convert results from IOB tags to spans across all teams and documents
output_spans = {
    t: [tp.iob_to_spans(df) for df in outputs[t]] for t in teams
}  # Type: Dict[str, List[pd.DataFrame]]

# As an example, show the first 10 spans that the "florian" team's model
# found on document 2.
output_spans["florian"][2].head(10)

In [ ]:
# Use Pandas merge to find what spans match up exactly for each team's
# results.
# Unlike Part 1, we perform the join across all entity types, looking for
# matches of both the extracted span *and* the entity type label.
def make_stats_df(gold_dfs, output_dfs):
    num_true_positives = [len(gold_dfs[i].merge(output_dfs[i]).index)
                          for i in range(len(gold_dfs))]
    num_extracted = [len(df.index) for df in output_dfs]
    num_entities = [len(df.index) for df in gold_dfs]
    doc_num = np.arange(len(gold_dfs))

    stats_by_doc = pd.DataFrame({
        "doc_num": doc_num,
        "num_true_positives": num_true_positives,
        "num_extracted": num_extracted,
        "num_entities": num_entities
    })
    stats_by_doc["precision"] = stats_by_doc["num_true_positives"] / stats_by_doc["num_extracted"]
    stats_by_doc["recall"] = stats_by_doc["num_true_positives"] / stats_by_doc["num_entities"]
    stats_by_doc["F1"] = 2.0 * (stats_by_doc["precision"] * stats_by_doc["recall"]) / (stats_by_doc["precision"] + stats_by_doc["recall"])
    return stats_by_doc

stats = {t: make_stats_df(gold_standard_spans, output_spans[t]) for t in teams}

# Show the result quality statistics by document for the "carrerasa" team
stats["carrerasa"]

In [ ]:
# F1 for document 4 is looking a bit low. Is that just fluke, or is it
# part of a larger trend? 
# In Part 1, we showed how to drill down to and examine "problem" documents.
# Since we have all this additional data, let's try a broader, more 
# quantitative approach. We'll start by building up some more fine-grained 
# data about congruence between the gold standard and the model outputs.
# Pandas' outer join will tell us what entities showed up just in the gold
# standard, just in the model output, or in both sets.
# For starters, let's do this just for the "carrerasa" team and document 4.
gold_standard_spans[4].merge(output_spans["carrerasa"][4], how="outer", indicator=True).sort_values("token_span")

In [ ]:
# Repeat the analysis from the previous cell across all teams and documents.
# That is, perform an outer join between the gold standard spans dataframe
# for each document and the corresponding dataframe from each team.
def merge_span_sets(team):
    result = []
    for i in range(len(gold_standard_spans)):
        merged = gold_standard_spans[i].merge(output_spans[team][i], how="outer", indicator=True)
        merged["gold"] = merged["_merge"].isin(("both", "left_only"))
        merged[team] = merged["_merge"].isin(("both", "right_only"))
        result.append(merged[["token_span", "ent_type", "gold", team]])
    return result

span_flags = {t: merge_span_sets(t) for t in teams}  # Type: Dict[str, List[pd.DataFrame]]

In [ ]:
# Now we have indicator variables for every extracted span, telling whether 
# it was in the gold standard data set and/or in each of the team's results.
# For example, here are the first 5 spans for document 2 in the "carrerasa"
# team's results:
span_flags["carrerasa"][2].head()

In [ ]:
# Do an n-way merge of all those indicator variables across documents.
# This operation produces a single summary dataframe per document.
indicators = []  # Type: List[pd.DataFrame]
for i in range(len(gold_standard_spans)):
    result = gold_standard_spans[i]
    for t in teams:
        result = result.merge(span_flags[t][i], how="outer")
    indicators.append(result.fillna(False))

In [ ]:
# Now we have a vector of indicator variables for every span extracted 
# from every document across all the model outputs and the gold standard.
# For example, let's show the results for document 10:
indicators[10]

In [ ]:
# If you look at the above dataframe, you can see that some entities 
# ("RUGBY UNION", for example) are "easy", in that almost every entry
# found them correctly. Other entities, like "CAMPESE", are "harder",
# in that few of the entrants correctly identified them. Let's add
# a column that quantifies this "difficulty level" by counting how 
# many teams found each true or false positive.
for df in indicators:
    # Convert the teams' indicator columns into a single matrix of 
    # Boolean values.
    vectors = df[df.columns[3:]].values
    counts = np.count_nonzero(vectors, axis=1)
    df["num_teams"] = counts

# Show the dataframe for document 10 again, this time with the new
# "num_teams" column at the far right.
indicators[10]

In [ ]:
# Now we can rank the entities in document 10 by "difficulty", either as 
# true positives for the models to find...
# (just for document 10 for the moment)
ind = indicators[10].copy()
ind[ind["gold"] == True].sort_values("num_teams").head(10)

In [ ]:
# ...or as false positives to avoid:
ind[ind["gold"] == False].sort_values("num_teams", ascending=False).head(10)

In [ ]:
# To get a better picture of what entities are "difficult", we need to look 
# across the entire validation set. Let's combine the dataframes in 
# `indicators` into a single dataframe that covers all the documents.

# First we preprocess each dataframe to make it easier to combine.
to_stack = [
    pd.DataFrame({
        "doc_id": i,
        # TokenSpanArrays from different documents can't currently be stacked,
        # so convert to TokenSpan objects.
        "token_span" : indicators[i]["token_span"].astype(object),
        "ent_type": indicators[i]["ent_type"],
        "gold": indicators[i]["gold"],
        "num_teams": indicators[i]["num_teams"]
    })
    for i in range(len(indicators))
]

# Then we concatenate all the preprocessed dataframes into a single dataframe.
all_counts = pd.concat(to_stack)

all_counts

In [ ]:
# Now we can pull out the most difficult entities across the entire validation
# set.
# First, let's find the most difficult entities from the standpoint of recall:
# entities that are in the gold standard, but not in most results.
difficult_recall = all_counts[all_counts["gold"] == True].sort_values("num_teams").reset_index(drop=True)
difficult_recall.head(10)

In [ ]:
# Hmm, everything is zero. How many entities were found by zero teams?  One team?
(all_counts[all_counts["gold"] == True][["num_teams", "token_span"]]
 .groupby("num_teams").count()
 .rename({"token_span": "count"}))
 # TODO: The last step here has no effect, due to a bug in Pandas. Fix the bug!

In [ ]:
# Yikes! 140 entities in the validation set were so hard to find, they
# were extracted by 0 teams.
# Let's go back and look at some of those 0-team entities in context:
difficult_recall["context"] = difficult_recall["token_span"].apply(lambda t: t.context())
pd.set_option('max_colwidth', 100)
difficult_recall.head(20)

**Some of these entities are "difficult" because the validation set contains incorrect labels.**

For reference, there's a copy of the CoNLL labeling rules in this repository at
[resources/conll_03/ner/annotation.txt](../resources/conll_03/ner/annotation.txt)

There are 4 incorrect labels in this first set of 20:
* `[3289, 3299): 'Full Light'` should be "Zywiec Full Light"
* `[11, 19): 'Honda RV'` should be tagged `ORG`
* `[1525, 1541): 'Consumer Project'` should be "Consumer Project on Technology" and should be tagged `ORG`
* `[244, 255): 'McDonald 's'` should be tagged `MISC` (because it's an "adjective ... derived from a word which is ... organisation")

In [ ]:
# Let's look at the entities that are difficult from the perspective of 
# precision: that is, in many models' results, but not in the gold standard.
difficult_precision = all_counts[all_counts["gold"] == False].sort_values("num_teams", ascending=False).reset_index(drop=True)

# Again, we can add some context to these spans:
difficult_precision["context"] = difficult_precision["token_span"].apply(lambda t: t.context())
difficult_precision.head(20)

**As with the entities in `difficult_recall`, some of these entities in `difficult_precision` are "difficult" because the validation set has missing and incorrect labels.**

**13** of these first 20 "incorrect" results are due to missing and incorrect labels:
* `[25, 32): 'BRITISH''` in document 202 should be tagged `MISC`.
* `[1317, 1327): 'Portsmouth'` in document 207 should be tagged `ORG`, not `LOC`.
* `[110, 118): 'Scottish'` in document 199 should be tagged `MISC`
  (or `[28, 53): 'SCOTTISH PREMIER DIVISION'` and 
  `[110, 135): 'Scottish premier division'` should both be tagged `ORG`).
* `[146, 163): 'Santiago Bernabeu'` in document 40 should be tagged `MISC`
  (because the "s" in `[146, 171): 'Santiago Bernabeu stadium'` is not capitalized).
* `[239, 251): 'Philadelphia'` in document 223 should be tagged `ORG`, not `LOC`.
* `[367, 376): 'Karlsruhe'` in document 36 should be tagged `ORG`, not `LOC`.
* `[1003, 1011): 'Congress'` in document 100 should be tagged `ORG`
  (also, `[957, 964): 'Chilean' ==> MISC` should be replaced with 
  `[957, 973): 'Chilean Congress' ==> ORG`).
* `[420, 428): 'Freiburg'` in document 36 should be tagged `ORG`, not `LOC`.
* In document 70, `[186, 211): 'New York Commodities Desk'`, not `[186, 206): 'New York Commodities'`, should be tagged `ORG`.
* `[263, 271): 'St Louis'` in document 223 should be tagged `ORG`, not `LOC`.
* `[788, 795): 'Antwerp'` in document 155 should be tagged `LOC`, not `ORG`.
* In document 112, `[178, 191): 'John Mills Jr'`, not `[178, 188): 'John Mills'`, should be tagged `PER`.
* `[274, 282): 'COLORADO'` in document 223 should be tagged `ORG`.


In [ ]:
# Here's the gold standard data for document 155, for example.
# Note line 12.
gold_standard_spans[155][0:60]

In [ ]:
# The above gold standard spans in context. 
gold_standard_spans[155]["token_span"].values